In [ ]:
import sys
import os
import json
from typing import Dict, List, Any
from pydantic import BaseModel

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"Added {project_root} to Python path")

from agent_system.agent_system import AgentSystem

# Initialize the agent system
agent_system = AgentSystem()

# Define function calling schema
class TaskPlan(BaseModel):
    steps: List[Dict[str, str]]
    assigned_agents: List[str]
    
def get_task_plan(task_description: str) -> Dict[str, Any]:
    """Function that will be available to LLM for planning tasks"""
    return {
        "name": "get_task_plan",
        "description": "Plan the execution steps and assign agents",
        "parameters": TaskPlan.model_json_schema()
    }

def assign_tasks(plan: Dict[str, Any]) -> str:
    """Distribute tasks to agents according to the plan"""
    # Distribute tasks to appropriate agents
    results = []
    for step in plan['steps']:
        agent = step['assigned_agent']
        task = step['task']
        results.append(f"{agent} assigned task: {task}")
    return "\n".join(results)

In [ ]:
system_message = """You are a Supervisor Agent responsible for planning and coordinating tasks.
Available Agents and their responsibilities:
1. Supervisor Agent (You): Project management and task coordination
2. Metadata Steward: Data governance and metadata management
3. Data Calibration: Data quality and validation
4. Data Developer: Data pipeline development and maintenance

When given a task, create a detailed plan and assign appropriate agents."""

# Define available functions
functions = [
    get_task_plan()
]

# Your prompt/task
user_prompt = """Create a data pipeline that:
1. Ingests customer transaction data
2. Validates data quality
3. Transforms it for analysis
4. Stores it in our data warehouse"""

# Call LLM with function calling
response = agent_system.llm.chat_completion(
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt}
    ],
    functions=functions,
    function_call={"name": "get_task_plan"}
)

# Extract and execute the plan
if response.choices[0].message.function_call:
    plan = json.loads(response.choices[0].message.function_call.arguments)
    result = assign_tasks(plan)
    print(result)